# Interesting Features of Haskell
- Types
- non-strictness
- recursion
- pattern matching
- Mini project!

## Mandatory helloWorld:

In [ ]:
main :: IO () -- [1] Type Sig
main = do -- [2] Function expression
  putStrLn "Hello World"

Executing:

In [ ]:
main

## Syntax:
- Expressions
- Type sigs

_Expressions_

In [ ]:
a = 1 + 2
b = "This is a String"

_Type Signatures:_
- _useful summaries of what functions can do_
- _structure your code_

In [ ]:
a :: Integer
b :: String

_Together_

In [ ]:
a :: Int
a = 1 + 2

b :: String
b = "This is a String"

## Types:
- String, Bool, Integer, Int
- More interesting types: 
  - Sum
  - Product
  - Maybe (if we have time)
  


#### Define your own types in Haskell:
- Create structure e.g. An object, a system etc...

## Sum Types

data Bool = True | False

In [ ]:
data Noel = Happy | Sad deriving (Eq, Show)

happy :: Noel
happy = Happy

sad :: Noel
sad = Sad

In [ ]:
main :: IO ()
main = do
  putStrLn $ show Happy
  putStrLn $ show Sad
  print $ Happy == Sad

_Executing_

In [ ]:
main

## Product Types

#### _Example: Modelling an Animal_

In [ ]:
data Animal = Animal Name Age Limbs Hair

type Name = String
type Age = Integer
type Limbs = Integer
data Hair = Hairy | Hairless deriving (Eq, Show)

## Functions in haskell

In [ ]:
add :: Int -> Int -> Int
--      [1]    [2]    [3]
add a b = a + b

**add** is a function which takes 2 Int args [1], [2] , returns an Int [3]

In [ ]:
addIO :: IO ()
addIO = print $ add 1 2

addIO

# Currying
To understand the way functions work lets talk about currying

functions in Haskell take one argument and return a function / value

In [ ]:
add :: Int -> (Int -> Int)
add a = \x -> a + x

-- (\x -> a + x) 5
-- (\x -> 6 + x) 5
-- (6 + 5)
-- 11

Hence, you can partially apply functions!

In [ ]:
partialAdd :: Int -> Int
partialAdd = add 5

partialAddIO :: IO ()
partialAddIO = do
  let result = partialAdd 6
  print result

_Execute:_

In [ ]:
partialAddIO

## Pattern matching

In [ ]:
const :: a -> b -> a
const a _ = a

sayHi :: String -> String
sayHi str =
  case str of
    "Noel" -> "Hello " ++ str
    "Harish" -> "Hello " ++ str
    _ -> "Who are you"
    
putStrLn $ sayHi "Noel"

## Recursion

Many things can be defined in Haskell through recursion

In [ ]:
length :: String -> Int
length "" = 0
length (x:xs) = 1 + length xs
--            = 1 + (1 + length xss)
--            = 1 + (1 + (1 + length xsss))

In [ ]:
import Control.Concurrent

main = do
  threadDelay 1000000
  putStrLn "Hello after 1000"
  
main

In [ ]:
type Path = String

path :: Path
path = "path"

view = "1" ++ path

In [ ]:
funcCheck :: Int -> Int -> Int
funcCheck start = 
  let middle = start + 5
  in \numbers -> middle + numbers

newtype State s a = State { runstate :: s -> (a, s)}

instance Functor (State s) where
  fmap f stateGenerate = State $ \state -> let (generated, _) = runstate stateGenerate $ state 
                                           in (f generated, state)
                                           
instance Applicative (State s) where
  pure a = State $ \state -> (a, state)
--(<*>) :: State s (a -> b) -> State s a -> State s b
  (<*>) stateF stateGenerate = State $ \state -> let (generated, _) = runstate stateGenerate $ state
                                                     (f, _) = runstate stateF $ generated
                                                 in (f generated, state)

instance Monad (State s) where
  return = pure
--(>>=) :: State s a -> (a -> State s b) -> State s b
  (>>=) stateGenerate liftToState = State $ \s0 -> let (generated, s2) = runstate stateGenerate $ s0
                                                      in runstate (liftToState generated) s2
-- (>>) :: m a -> m b -> m b  
  (>>) state1 state2 = state1 >>= \_ -> state2
-- stateGenerate = \state -> (a, state)

put newState = State $ \_ -> ((), newState) -- sets the state value
get = State $ \s -> (s, s) -- sets the state value to result value
return a = State $ \s -> (a, s) -- sets the result value

evalState :: State s a -> s -> a
evalState p s = fst $ runstate p s
execState :: State s a -> s -> s
execState p s = snd $ runstate p s

Recursively downloading a google drive Folder

In [ ]:
data Comb a b = Comb { runComb :: s -> (a, s) }
print $ get (Comb "me" 5)
type Comb' = Comb String String
zex = Comb "Asd" "asd" :: Comb'
print $ get zex

-- line numbers offset by +7
-- mocks
-- Behaviour: When 

In [1]:
import Control.Concurrent
-- threadDelay 1000000

import System.Random
import Control.Monad.State

type Name = String
type MimeType = String
type Id = String
-- data State s a = State { runState :: s -> (a, s)}
newtype File = File Name deriving (Eq, Show)

data Folder = Directory { getDirName :: Name
                        , getDirFiles :: [File]
                        , getDirFolders :: [Folder] } deriving Eq
 
con :: Name -> Name -> Name
con = (++)
branch :: Name -> Name
branch a = "  |" `con` "\n  --" `con` a `con` "\n"

indentation :: Int -> String
indentation 0 = ""
indentation n = indentation (n - 1) ++ "  "

indentBranch :: Int -> Name -> Name
indentBranch indent a = concat [indentation indent, "|", "\n", indentation indent, "--", a, "\n"]

{-
instance Show Folder where
  show dir = go dir 0 where
    go dir indent = 
        case dir of
          Directory name [] [] -> "Directory" `con` name
          Directory name fLs dLs -> "Directory " `con` name `con` "\n"
                                    `con` concat (fmap (branch . show) fLs)
                                    `con` concat (fmap (branch . show) dLs)
-}

instance Show Folder where
  show dir = go dir 0 where
    go dir indent = 
        case dir of
          Directory name fLs dLs -> indentBranch indent ("Directory " `con` name `con` "\n")
                                    `con` concat (fmap ((indentBranch (indent + 1)) . show) fLs)
                                    `con` concat (fmap (\dir -> go dir (indent + 1)) dLs)
            
initialState :: Folder -- State Folder Folder
initialState = Directory "root" [] [] -- State $ \folder -> (Dire root, folder)

biggerDir :: Folder
biggerDir = Directory 
              "root2" 
              [File "a", File "b"] 
              [Directory 
                "root3" 
                [File "a", File "b"]
                []]

biggestDir :: Folder
biggestDir = Directory 
              "root2" 
              [File "a", File "b"] 
              [Directory 
                "root3" 
                [File "a", File "b"]
                [],
               Directory
                "root4"
                [File "a", File "c"]
                [Directory
                  "root5"
                  [File "d"]
                  []]]


type Path = [Name]
-- define show
addFileRoot :: File -> Folder -> Folder
addFileRoot file dir = 
  Directory (getDirName dir) 
            (file : getDirFiles dir) 
            (getDirFolders dir)

addFolderRoot :: Folder -> Folder -> Folder
addFolderRoot folder dir = 
  Directory (getDirName dir) 
            (getDirFiles dir) 
            (folder : getDirFolders dir)

modFolder :: (Folder -> Folder) -> Path -> Folder -> Folder
modFolder f [] folder = folder
modFolder f (x:xs) folder
  | x == folderName = 
    case xs of
      [] -> f folder
      xss -> Directory folderName fileList (fmap (modFolder f xs) folderList)
  | otherwise = folder
  where (folderName, 
         fileList, 
         folderList) = (getDirName folder, 
                        getDirFiles folder,
                        getDirFolders folder)
                        
newFolder = addFolderRoot biggerDir initialState

--modFolder (addFileRoot (File "B1")) ["root", "root2"] newFolder
--modFolder (addFileRoot (File "B2")) [""] newFolder
--modFolder (addFolderRoot newFolder) ["root", "root2", "root3"] newFolder

In [2]:
type Local = State Folder Folder

In [3]:
data Target = Target { getTargetName :: Name, 
                       getTargetMime :: MimeType,
                       getTargetId :: Id,
                       getParentId :: Id} deriving (Eq, Show)

mockFilesDb :: [Target]
mockFilesDb = [
  Target { getTargetName = "Root", 
           getTargetMime = "Folder", 
           getTargetId = "A1",  
           getParentId = "NOOOO" },
  Target { getTargetName = "UserFiles",
           getTargetMime = "Folder",
           getTargetId = "B1",
           getParentId = "A1" },
  Target { getTargetName = "hello.Docx",
           getTargetMime = "Doc",
           getTargetId = "A2",
           getParentId = "A1" },
  Target { getTargetName = "details.xlsx",
           getTargetMime = "Spreadsheet",
           getTargetId = "A3",
           getParentId = "A1" },
  Target { getTargetName = "DataFiles",
           getTargetMime = "Folder",
           getTargetId = "C1",
           getParentId = "B1" },
  Target { getTargetName = "log.xlsx",
           getTargetMime = "Spreadsheet",
           getTargetId = "C2",
           getParentId = "C1" }]
           

In [4]:
main :: State Int Int
main = do
  og <- state $ \x -> (5, 2 * x)
  state $ \y -> (og, 2 * y)
  state $ \z -> (5, z)
  state $ \d -> (10, 5 * d)
  x <- get
  return x
  
  
runState main 5

(100,100)

In [5]:
fileDownload :: Id -> Path -> Local
fileDownload ident path = do
  originalDirectory <- get
  let file :: [Target]
      file = filter (\target -> getTargetId target == ident) mockFilesDb
  case file of
    [] -> state $ \_ -> (originalDirectory, originalDirectory) -- State { runState :: Localstate -> (LocalChangeable, Localstate)}
    (x:xs) -> do
      let fileLocal :: File
          fileLocal = File $ getTargetName x
          stateModFunc :: Folder -> Folder
          stateModFunc = modFolder (\rootFolder -> (addFileRoot fileLocal rootFolder))
                                   path
      state $ \folder -> (folder, stateModFunc folder)

flowState = fileDownload "A2" ["root","root2"]

twoSame = flowState >> flowState
execState twoSame newFolder
evalState flowState newFolder

|
--Directory root

  |
  --Directory root2

    |
    --File "hello.Docx"
    |
    --File "hello.Docx"
    |
    --File "a"
    |
    --File "b"
    |
    --Directory root3

      |
      --File "a"
      |
      --File "b"

|
--Directory root

  |
  --Directory root2

    |
    --File "a"
    |
    --File "b"
    |
    --Directory root3

      |
      --File "a"
      |
      --File "b"

In [6]:
folderFrom :: Name -> Path -> Local
folderFrom name path = do
  originalDirectory <- get
  let emptyDir :: Folder
      emptyDir = Directory name [] []
      stateModFunc :: Folder -> Folder
      stateModFunc = modFolder (\rootFolder -> (addFolderRoot emptyDir rootFolder))
                                               path
  state $ \folder -> (folder, stateModFunc folder)

flowFolder = folderFrom "Noel" ["root", "root2", "root3"]

execState flowFolder newFolder

|
--Directory root

  |
  --Directory root2

    |
    --File "a"
    |
    --File "b"
    |
    --Directory root3

      |
      --File "a"
      |
      --File "b"
      |
      --Directory Noel

In [7]:
getFolderList :: Id -> [Target]
getFolderList ident =
  let files = filter (\target -> getParentId target == ident) mockFilesDb
  in case files of
      [] -> []
      xs -> xs

getFolderList "A1"
getFolderList "B1"
getFolderList "C1"

[Target {getTargetName = "UserFiles", getTargetMime = "Folder", getTargetId = "B1", getParentId = "A1"},Target {getTargetName = "hello.Docx", getTargetMime = "Doc", getTargetId = "A2", getParentId = "A1"},Target {getTargetName = "details.xlsx", getTargetMime = "Spreadsheet", getTargetId = "A3", getParentId = "A1"}]

[Target {getTargetName = "DataFiles", getTargetMime = "Folder", getTargetId = "C1", getParentId = "B1"}]

[Target {getTargetName = "log.xlsx", getTargetMime = "Spreadsheet", getTargetId = "C2", getParentId = "C1"}]

In [8]:
--inform :: String -> IO ()
--inform msg = putStrLn msg

seqMap :: (a -> Local) -> [a] -> Local
seqMap _ [] = get
seqMap f (x:xs) = f x >> seqMap f xs
--struggles: query rate

In [ ]:
targetedFold :: Target
targetedFold = Target { getTargetName = "Root", 
                          getTargetMime = "Folder", 
                          getTargetId = "A1",  
                          getParentId = "" }

                          
handles :: Target -> Local
handles target
  | targetMime == "Spreadsheet" || 
    targetMime == "Doc" = getFile target (["root"] ++ [folderName])
  | targetMime == "Folder" = mkFolder target (["root"])
  where folderName = getTargetName target
        targetMime = getTargetMime target
        
flowBasic = handles targetedFold

execState flowBasic newFolder

In [9]:
-- data File = File { getFileName :: Name, getMime :: MimeType }
-- data Folder = Folder { getFolderName :: Name }



-- newtype Download a = Download (IO a)
-- newtype MakeFolder a = MakeFolder (IO a)

mimeCheck :: [MimeType] -> MimeType -> Maybe MimeType
mimeCheck [] _ = Nothing
mimeCheck (mimeType : mimeTypes) sampleMimeType
  | mimeType == sampleMimeType = Just mimeType
  | otherwise = mimeCheck mimeTypes sampleMimeType
  
getFile :: Target -> Path -> Local
getFile target = fileDownload $ getTargetId target

mkFolder :: Target -> Path -> Local
mkFolder target = folderFrom $ getTargetName target
  
listFolder :: Target -> [Target]
listFolder target = getFolderList $ getTargetId target

getFolder :: Target -> Path -> Local
getFolder target [] = get
getFolder target path@(x:xs) = do
  originalState <- get
  let targetMime :: MimeType
      targetMime = getTargetMime target
  let mimes :: [MimeType]
      mimes = ["Folder","Doc", "Spreadsheet"]
  case mimeCheck mimes targetMime of
    Just "Folder" -> do
      mkFolder target path
      let folderName = getTargetName target
      let subFiles = listFolder target
      case subFiles of
        [] -> get
        targets -> seqMap handle targets
          where handle :: Target -> Local
                handle target 
                  | targetMime == "Spreadsheet" || 
                    targetMime == "Doc" = getFile target (path ++ [folderName])
                  | targetMime == "Folder" = getFolder target (path ++ [folderName])
                  where targetMime = getTargetMime target
                  
    _ -> state $ \_ -> (originalState, originalState)

targetedFolder :: Target
targetedFolder = Target { getTargetName = "Root", 
                          getTargetMime = "Folder", 
                          getTargetId = "A1",  
                          getParentId = "" }

flowFromDb :: Local
flowFromDb = getFolder targetedFolder ["root"]

main :: Folder
main =
  execState flowFromDb newFolder

newFolder

main

|
--Directory root

  |
  --Directory root2

    |
    --File "a"
    |
    --File "b"
    |
    --Directory root3

      |
      --File "a"
      |
      --File "b"

|
--Directory root

  |
  --Directory Root

    |
    --File "details.xlsx"
    |
    --File "hello.Docx"
    |
    --Directory UserFiles

      |
      --Directory DataFiles

        |
        --File "log.xlsx"
  |
  --Directory root2

    |
    --File "a"
    |
    --File "b"
    |
    --Directory root3

      |
      --File "a"
      |
      --File "b"

## Combining Recursion, Sum, Product Types

In [ ]:
data List a = Empty | Cons a (List a) deriving (Eq, Show)